# Serbian Banknote Recognition
**Autor:** Stefan Stjepić SV40/2022  
**Predmet:** Računarska inteligencija  
**Cilj:** Razvoj sistema koji **automatski prepoznaje srpske novčanice** kombinacijom *detekcije (YOLOv8)* i *klasifikacije (CNN)*.

Ovaj notebook demonstrira ceo tok projekta:
1. Priprema i pretprocesiranje dataset-a za treniranje YOLOv8 modela  
2. Treniranje YOLOv8 modela za detekciju novčanica  
3. Priprema i pretprocesiranje dataset-a za treniranje CNN modela 
4. Treniranje CNN modela za klasifikaciju apoena  
5. Kombinovanje YOLO + CNN modela za prepoznavanje srpskih novčanica

## 1. Priprema i pretprocesiranje dataset-a za treniranje YOLOv8 modela

Dataset: [Serbian Banknotes - Kaggle](https://www.kaggle.com/datasets/stefanstjepic/serbian-banknotes)

Sadrži:
- 270 slika -> 30 po apoenu [10, 20, 50, 100, 200, 500, 1000, 2000, 5000], 15 po strani
- YOLO label -> .txt fajlovi koji sadrže podatke oblika
```bash
        <class_id> <x_center> <y_center> <width> <height>
```

Koraci obrade:
- Resize na 512x512
- Podela na train/val/test (70/15/15)

### Skripta za pretprocesiranje

In [23]:
!python src/yolo/yolo_preprocess.py

1/270
2/270
3/270
4/270
5/270
6/270
7/270
8/270
9/270
10/270
11/270
12/270
13/270
14/270
15/270
16/270
17/270
18/270
19/270
20/270
21/270
22/270
23/270
24/270
25/270
26/270
27/270
28/270
29/270
30/270
31/270
32/270
33/270
34/270
35/270
36/270
37/270
38/270
39/270
40/270
41/270
42/270
43/270
44/270
45/270
46/270
47/270
48/270
49/270
50/270
51/270
52/270
53/270
54/270
55/270
56/270
57/270
58/270
59/270
60/270
61/270
62/270
63/270
64/270
65/270
66/270
67/270
68/270
69/270
70/270
71/270
72/270
73/270
74/270
75/270
76/270
77/270
78/270
79/270
80/270
81/270
82/270
83/270
84/270
85/270
86/270
87/270
88/270
89/270
90/270
91/270
92/270
93/270
94/270
95/270
96/270
97/270
98/270
99/270
100/270
101/270
102/270
103/270
104/270
105/270
106/270
107/270
108/270
109/270
110/270
111/270
112/270
113/270
114/270
115/270
116/270
117/270
118/270
119/270
120/270
121/270
122/270
123/270
124/270
125/270
126/270
127/270
128/270
129/270
130/270
131/270
132/270
133/270
134/270
135/270
136/270
137/270
138/270
139/

Kada se skripta izvrši, kreiraće se folder *images* unutar *data/processed* koji će sadržati slike dimenzija 512x512.

### Skripta za podelu dataset-a

In [24]:
!python src/yolo/yolo_split_dataset.py

✅ Dataset podeljen na train/val/test foldere.
Train: 189, Val: 40, Test: 41


Kada se skripta izvrši, kreiraće se folderi *train, val i test* unutar *data/processed* koji će sadržati podfoldere *images* i *labels*. U *images* folderu će se nalaziti slike dok će odgovarajuće labele biti u *labels* folderu.

## 2. Treniranje YOLOv8 modela za detekciju novčanica  
Cilj treniranja YOLOv8 modela jeste da model bude sposoban da detektuje prisustvo novčanice na slici kao i da odredi gde se ona nalazi.  
Konfiguracija za treniranje modela se nalazi unutar *configs/data.yaml fajla*.
```bash
    nc: 1

    names: ["banknote"]

    train: ../data/processed/train/images
    val: ../data/processed/val/images
    test: ../data/processed/test/images
```

### Skripta za treniranje modela

In [ ]:

!python src/yolo/yolo_train.py

Kada se model istrenira, bice sačuvan unutar *runs/train/serbian_banknotes/weignts/best.pt*.

### Skripte za testiranje modela

Postoje 2 skripte za testiranje. Jedna skripta ispisuje metrike modela dok druga vrši vizuelno testiranje tako što prikazuje slike sa novčanicama i na svakoj pokaže svoju predikciju gde se nalazi.

In [ ]:
# skripta koja prikazuje vednost metrika
!python src/yolo/yolo_test.py

```bash
=== Evaluation Metrics ===
Precision (P):   0.999
Recall (R):      1.000
mAP@0.5:         0.995
mAP@0.5:0.95:    0.921
```

In [ ]:
# skripta za vizuelno testiranje
!python src/yolo/yolo_test_bb.py

## 3. Priprema i pretprocesiranje dataset-a za treniranje CNN modela 

Pomoću našeg YOLO modela radimo kropovanje slika slika iz originalnog data seta koji se nalazi u *data/raw/images* i obrađene slike smeštamo unutar *data/processed/cnn/cnn_dataset*

In [ ]:
!python src/yolo/yolo_crop_images.py

Sada imamo dataset unutar *data/processed/cnn/cnn_dataset* koji je neophodno podeliti u podfoldere po vrednosti apoena a zatim podeliti na train/test/val grupe da bismo mogli da treniramo model.

In [ ]:
!python src/cnn/cnn_sort_dataset.py

In [ ]:
!python src/cnn/cnn_split_dataset.py

Ako je sve uspešno izvršeno, unutar *data/processed/cnn* foldera će se pojaviti *cnn_sorted* i *train, test i val* podfolderi.

## 4. Treniranje CNN modela za klasifikaciju apoena  

Cilj treniranja CNN modela jeste dobiti model koji je sposoban da klasifikuje novčanice na osnovu vrednosti novčanice.

### Konfiguracije 
``` bash
{
  "data": {
    "train_dir": "./data/processed/cnn/train",
    "val_dir": "./data/processed/cnn/val",
    "test_dir": "./data/processed/cnn/test"
  },
  "model": {
    "save_dir": "cnn_model",
    "weights_file": "serbian_banknote_cnn.pth",
    "img_size": 256
  },
  "classes": ["10", "20", "50", "100", "200", "500", "1000", "2000", "5000"],
  "training": {
    "batch_size": 32,
    "lr": 0.001,
    "num_epochs": 50
  }
}
```

### Model
``` bash
class SerbianBanknoteCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.dropout = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(128*30*30, 256) 
        self.fc2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = self.feature_extraction(x)        
        x = x.view(x.size(0), -1)              
        x = self.dropout(F.relu(self.fc1(x)))   
        x = self.fc2(x)                         
        return x

```


In [ ]:
!python src/cnn/cnn_train.py

```bash
Epoch [1/50] Train Loss: 2.8038 Val Loss: 2.1908 Val Acc: 11.11%
✅ Best model saved at epoch 1 with Val Acc: 11.11%
Epoch [2/50] Train Loss: 2.1706 Val Loss: 2.0786 Val Acc: 16.67%
✅ Best model saved at epoch 2 with Val Acc: 16.67%
Epoch [3/50] Train Loss: 2.0407 Val Loss: 1.8415 Val Acc: 38.89%
✅ Best model saved at epoch 3 with Val Acc: 38.89%
Epoch [4/50] Train Loss: 1.7591 Val Loss: 1.5717 Val Acc: 47.22%
✅ Best model saved at epoch 4 with Val Acc: 47.22%
Epoch [5/50] Train Loss: 1.3322 Val Loss: 1.0311 Val Acc: 55.56%
✅ Best model saved at epoch 5 with Val Acc: 55.56%
Epoch [6/50] Train Loss: 0.9976 Val Loss: 1.2348 Val Acc: 61.11%
✅ Best model saved at epoch 6 with Val Acc: 61.11%
Epoch [7/50] Train Loss: 0.7860 Val Loss: 0.6087 Val Acc: 69.44%
✅ Best model saved at epoch 7 with Val Acc: 69.44%
Epoch [8/50] Train Loss: 0.5197 Val Loss: 0.5022 Val Acc: 77.78%
✅ Best model saved at epoch 8 with Val Acc: 77.78%
Epoch [9/50] Train Loss: 0.4516 Val Loss: 0.5131 Val Acc: 80.56%
✅ Best model saved at epoch 9 with Val Acc: 80.56%
Epoch [10/50] Train Loss: 0.1987 Val Loss: 0.5757 Val Acc: 75.00%
Epoch [11/50] Train Loss: 0.1575 Val Loss: 0.5002 Val Acc: 83.33%
✅ Best model saved at epoch 11 with Val Acc: 83.33%
Epoch [12/50] Train Loss: 0.1479 Val Loss: 0.4001 Val Acc: 91.67%
✅ Best model saved at epoch 12 with Val Acc: 91.67%
Epoch [13/50] Train Loss: 0.0769 Val Loss: 0.5142 Val Acc: 80.56%
Epoch [14/50] Train Loss: 0.0504 Val Loss: 0.4931 Val Acc: 88.89%
...
Epoch [47/50] Train Loss: 0.0015 Val Loss: 0.5832 Val Acc: 88.89%
Epoch [48/50] Train Loss: 0.0015 Val Loss: 0.5781 Val Acc: 88.89%
Epoch [49/50] Train Loss: 0.0011 Val Loss: 0.5734 Val Acc: 88.89%
Epoch [50/50] Train Loss: 0.0041 Val Loss: 0.5855 Val Acc: 86.11%
```

Posle uspešno završenog testiranja CNN model će biti sačuvan u *cnn_model/* folderu kao *serbian_banknote_cnn.pth*

### Skripta za prikaz metrika

In [ ]:
!python src/cnn/cnn_test.py

## 5. Kombinovanje YOLO + CNN modela za prepoznavanje srpskih novčanica

Sada je na redu da se testira kako ova dva modela rade zajedno. Za to je neophodno pokrenuti *main.py* u terminalu kao što je napisano.
### Tok:
- učitavamo yolo i cnn modele
- unosimo putanju do slike novčanice
- dobijamo predikciju


In [ ]:
!python src/main.py